In [1]:
import torch
import numpy as np
import random
seed=1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [2]:
from transformers import AutoTokenizer
import sacrebleu
import sys
import os
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, MBartForConditionalGeneration, MBart50TokenizerFast

In [3]:
from datasets import load_dataset, load_metric

metric = load_metric("sacrebleu")

In [4]:
tgt_lang_to_code = {
    "hi" : "hi_IN",
    "de" : "de_DE",
    "es" : "es_XX",
    "it" : "it_IT",
    "ru" : "ru_RU",
    "ja" : "ja_XX"
}

In [ ]:
def read_file(fname):
    data = []
    with open(fname) as f:
        for line in f:
            data.append(line.strip())
    return data

def get_data(tgt_lang, domain, split):
    source = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.en")
    formal_translations = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.formal.{tgt_lang}")
    informal_translations = read_file(f"../internal_split/en-{tgt_lang}/{split}.{domain}.informal.{tgt_lang}")
    return source, formal_translations, informal_translations

In [ ]:
MAX_LENGTH=64
model_name="facebook/mbart-large-50-one-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/fs/clip-scratch/sweagraw/CACHE")

In [ ]:
import torch
from torch.utils.data import Dataset

def collate_fn(batch, collator, mask_prob, mask_idx):
    batch = collator(batch)
    mask = torch.randn_like(batch["covariate_ids"], dtype=torch.float32) < mask_prob
    batch["covariate_ids"] = batch["covariate_ids"].masked_fill(mask, mask_idx).unsqueeze(0).T
    return batch

class FormalityData(Dataset):
    
    def __init__(self, domain, split, src_lang, tgt_lang):
        self.source, self.formal_translations, self.informal_translations=get_data(tgt_lang, domain, split)
        tokenizer.src_lang = "en-XX"
        tokenizer.tgt_lang  = tgt_lang_to_code[tgt_lang]
        self.max_target_length=MAX_LENGTH
        self.max_input_length=MAX_LENGTH
        self.model_inputs, self.model_outputs = self.encode_split()
        self.tgt_lang = tgt_lang
        
    def __len__(self):
        return len(self.model_inputs["input_ids"])
    
    def encode_split(self):
        model_inputs = tokenizer(self.source, max_length=self.max_input_length, truncation=True)
        model_outputs = {}
        with tokenizer.as_target_tokenizer():
            model_outputs["formal"] = tokenizer(self.formal_translations, max_length=self.max_target_length, truncation=True)
            model_outputs["informal"] = tokenizer(self.informal_translations, max_length=self.max_target_length, truncation=True)
        return model_inputs, model_outputs

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.model_inputs.items()}
        if random.random() < 0.5:
            item["labels"] = self.model_outputs["formal"]["input_ids"][idx]
            item["covariate_ids"] = torch.tensor([0]).unsqueeze(0).T
        else:
            item["labels"] = self.model_outputs["informal"]["input_ids"][idx]
            item["covariate_ids"] = torch.tensor([1]).unsqueeze(0).T
        item["forced_bos_token_id"] = tokenizer.lang_code_to_id[tgt_lang_to_code[self.tgt_lang]]
        return item

In [ ]:
domain="combined"
src_lang="en"
direction="informal"
train_datasets = []
for tgt_lang in ["hi", "ja", "de", "es"]:
    train_datasets.append(FormalityData(domain, "train", src_lang, tgt_lang))
train_dataset = torch.utils.data.ConcatDataset(train_datasets)

dev_datasets = []
for tgt_lang in ["hi", "ja", "de", "es"]:
    dev_datasets.append(FormalityData(domain, "dev", src_lang, tgt_lang))
dev_dataset = torch.utils.data.ConcatDataset(dev_datasets)

In [ ]:
from mbart_covariate import CMBartForConditionalGeneration, MBartSeq2SeqTrainer

In [ ]:
model_name="facebook/mbart-large-50-one-to-many-mmt"

In [ ]:
model_name="../models/facebook/mbart-large-50-one-to-many-mmt-finetuned-covariate-en-to-xx/checkpoint-500"

In [ ]:
model = CMBartForConditionalGeneration.from_pretrained(model_name, cache_dir="/fs/clip-scratch/sweagraw/CACHE",num_covariates=2)

In [ ]:
model.requires_grad_(False)
model.get_covariate().requires_grad_(True)
# model.get_decoder().requires_grad_(True)
model.get_decoder().layers[-2:].requires_grad_(True)
model.lm_head.requires_grad_(True)

In [ ]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Training {count_params(model)} parameters...")

In [ ]:
train_batch_size = 1
eval_batch_size=1
MASK_PROB = 0.0
style_mask=model.config.style_mask
args = Seq2SeqTrainingArguments(
    output_dir=f"../models/{model_name}-finetuned-covariate-{src_lang}-to-xx",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    weight_decay=0.01,
    num_train_epochs=10,
    eval_steps=100,
    gradient_accumulation_steps=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False
)

In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = MBartSeq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=lambda batch: collate_fn(batch, collator, 0.0,  model.config.style_mask),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()